<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/peshitta_small.png"/>
<img align="right" src="images/tf-small.png"/>


# Tutorial

This notebook gets you started with using
[Text-Fabric](https://github.com/Dans-labs/text-fabric) for coding in Syriac texts.

Chances are that a bit of reading about the underlying
[data model](https://github.com/Dans-labs/text-fabric/wiki/Data-model)
helps you to follow the exercises below, and vice versa.

Most programs start with loading a few modules.
In the next cell, the first line loads standard modules that come with Python itself,
and the second cell loads Text-Fabric.

Before you can run this, you need to install it.
The basic instruction for that is, on a terminal:

```
pip install text-fabric
```

if you have installed Python with the help of Anaconda, or

```
sudo -H pip3 install text-fabric
```
if you have installed Python from [python.org](https://www.python.org).

Make sure that you do all this with Python **3**, not 2.

In [1]:
import sys, os, collections
from tf.fabric import Fabric

# Call Text-Fabric

Everything starts by setting up Text-Fabric.
It needs to know where to look for data.

The Syriac texts are in the same repository as this tutorial.
I assume you have cloned [linksyr](https://github.com/etcbc/linksyr).
in your directory `~/github/etcbc`, so that your directory structure looks like this

    your home direcectory\
    |                     - github\
    |                       |      - etcbc\
    |                       |        |         - linksyr
    
## Tip
If you start computing with this tutorial, first copy its parent directory to somewhere else,
outside your `linksyr` directory.
If you pull changes from the `linksyr` repository later, your work will not be overwritten.
Where you put your tutorial directory is up till you.
It will work from any directory.

In [2]:
REPO = '~/github/etcbc/linksyr'
SOURCE = 'syrnt'
CORPUS = f'{REPO}/data/tf/{SOURCE}'
TF = Fabric(locations=[CORPUS], modules=[''], silent=False )

This is Text-Fabric 3.1.5
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

30 features found and 0 ignored


# Load Features
The data of the corpus is organized in features.
They are *columns* of data.
Think of the text as a gigantic spreadsheet, where row 1 corresponds to the
first word, row 2 to the second word, and so on, for all 100,000+ words.

The letters of each word is a column `form` in that spreadsheet.

The corpus contains ca. 30 columns, not only for the words, but also for 
textual objects, such as *books*, *chapters*, and *verses*.

Instead of putting that information in one big table, the data is organized in separate columns.
We call those columns **features**.

We just load the features we need for this tutorial.
Later on, where we use them, it will become clear what they mean.

In [3]:
api = TF.load('''
    form
    grammatical_category
    lexeme
''')
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.04s B form                 from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.04s B grammatical_category from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.05s B lexeme               from /Users/dirk/github/etcbc/linksyr/data/tf/syrnt
   |     0.00s Feature overview: 28 for nodes; 1 for edges; 1 configs; 7 computed
  0.28s All features loaded/computed - for details use loadLog()


The result of this all is that we have a bunch of special variables at our disposal
that give us access to the text and data of the tablets.

At this point it is helpful to throw a quick glance at the text-fabric
[API documentation](https://github.com/Dans-labs/text-fabric/wiki/Api)
especially the right side bar.

The most essential thing for now is that we can use `F` to access the data in the features
we've loaded.
But there is more, such as `N`, which helps us to walk over the text, as we see in a minute.

# Counting

In order to get acquainted with the data, we start with the simple task of counting.

## Count all nodes
We use the 
[`N()` generator](https://github.com/Dans-labs/text-fabric/wiki/Api#walking-through-nodes)
to walk through the nodes.

We compared the tablet data to a gigantic spreadsheet, where the rows correspond to the words.
In Text-Fabric, we call the rows `slots`, because they are the textual positions that can be filled with words.

We also mentioned that there are also other textual objects. 
They are the tablets, columns, lines, etc.
They also correspond to rows in the big spreadsheet.

In Text-Fabric we call all these rows *nodes*, and the `N()` generator
carries us through those nodes in the textual order.

Just one extra thing: the `info` statements generate timed messages.
If you use them instead of `print` you'll get a sense of the amount of time that 
the various processing steps typically need.

In [4]:
indent(reset=True)
info('Counting nodes ...')

i = 0
for n in N(): i += 1

info('{} nodes'.format(i))

  0.00s Counting nodes ...
  0.04s 117884 nodes


Here you see it: more than 100,000 nodes!

## What are all those nodes?
Every node has a type, like sign, or line, face.
We know that we have many of them,
but what exactly are they?

Text-Fabric has two special features, `otype` and `oslots`, that must occur in every Text-Fabric data set.
`otype` tells you for each node its type, and you can ask for the number of `slot`s in the text.

Here we go!

In [5]:
F.otype.slotType

'word'

In [6]:
F.otype.maxSlot

109640

In [7]:
F.otype.maxNode

117884

In [8]:
F.otype.all

('book', 'chapter', 'verse', 'word')

In [9]:
C.levels.data

(('book', 4060.740740740741, 109641, 109667),
 ('chapter', 421.6923076923077, 109668, 109927),
 ('verse', 13.779062460726404, 109928, 117884),
 ('word', 1, 1, 109640))

This is interesting: above you see all the textual objects, with the average size of their objects,
the node where they start, and the node where they end.

## Count individual object types
This is an intuitive way to count the number of nodes in each type.
Note in passing, how we use the `indent` in conjunction with `info` to produce neat timed 
and indented progress messages.

In [10]:
indent(reset=True)
info('counting objects ...')

for otype in F.otype.all:
    i = 0

    indent(level=1, reset=True)

    for n in F.otype.s(otype): i+=1

    info('{:>7} {}s'.format(i, otype))

indent(level=0)
info('Done')

  0.00s counting objects ...
   |     0.00s      27 books
   |     0.00s     260 chapters
   |     0.00s    7957 verses
   |     0.03s  109640 words
  0.04s Done


# Feature statistics

`F`
gives access to all features.
Every feature has a method
`freqList()`
to generate a frequency list of its values, higher frequencies first.
Here are the lexemes (the top 20):

In [11]:
F.lexeme.freqList()[0:20]

(('L', 4234),
 ('HOA', 4006),
 ('HO', 3397),
 ('MN', 3335),
 ('LA', 3140),
 ('AMR', 2553),
 ('D;N', 1828),
 ('EL', 1813),
 ('ANA', 1728),
 ('HNA', 1584),
 ('ANT', 1401),
 ('CL', 1399),
 ('ALHA', 1389),
 ('CD', 1214),
 (';WOE', 1115),
 ('A;T', 1100),
 ('G;R', 1085),
 ('ATA', 1047),
 ('A;NA', 858),
 ('B', 824))

# Layer API
We travel upwards and downwards, forwards and backwards through the nodes.
The Layer-API (`L`) provides functions: `u()` for going up, and `d()` for going down,
`n()` for going to next nodes and `p()` for going to previous nodes.

These directions are indirect notions: nodes are just numbers, but by means of the
`oslots` feature they are linked to slots. One node *contains* an other node, if the one is linked to a set of slots that contains the set of slots that the other is linked to.
And one if next or previous to an other, if its slots follow of precede the slots of the other one.

`L.u(node)` **Up** is going to nodes that embed `node`.

`L.d(node)` **Down** is the opposite direction, to those that are contained in `node`.

`L.n(node)` **Next** are the next *adjacent* nodes, i.e. nodes whose first slot comes immediately after the last slot of `node`.

`L.p(node)` **Previous** are the previous *adjacent* nodes, i.e. nodes whose last slot comes immediately before the first slot of `node`.

All these functions yield nodes of all possible otypes.
By passing an optional parameter, you can restrict the results to nodes of that type.

The result are ordered according to the order of things in the text.

The functions return always a tuple, even if there is just one node in the result.

## Going up
We go from the first sign to the tablet it contains.
Note the `[0]` at the end. You expect one book, yet `L` returns a tuple. 
To get the only element of that tuple, you need to do that `[0]`.

If you are like me, you keep forgetting it, and that will lead to weird error messages later on.

In [12]:
firstBook = L.u(1, otype='book')[0]
print(firstBook)

109641


In [13]:
T.sectionFromNode(firstBook)

('Matt', 1, 1)

And let's see all the containing objects of word 100:

In [14]:
w = 100
for otype in F.otype.all:
    if otype == F.otype.slotType: continue
    up = L.u(w, otype=otype)
    upNode = None if len(up) == 0 else up[0]
    if upNode is None:
        print('word {} is not contained in a {}'.format(w, otype))
    else:
        print('word {} is contained in {} {}'.format(w, otype, upNode))

word 100 is contained in book 109641
word 100 is contained in chapter 109668
word 100 is contained in verse 109938


## Going next
Let's go to the next nodes of the first book.

In [16]:
afterFirstBook = L.n(firstBook)
for n in afterFirstBook:
    print('{:>7}: {:<13} first slot={:<6}, last slot={:<6}'.format(
        n, F.otype.v(n),
        E.oslots.s(n)[0],
        E.oslots.s(n)[-1],
    ))
secondBook = L.n(firstBook, otype='book')[0]

  13980: word          first slot=13980 , last slot=13980 
 110999: verse         first slot=13980 , last slot=13985 
 109696: chapter       first slot=13980 , last slot=14490 
 109642: book          first slot=13980 , last slot=22772 


## Going previous

And let's see what is right before the second book.

In [17]:
for n in L.p(secondBook):
    print('{:>7}: {:<13} first slot={:<6}, last slot={:<6}'.format(
        n, F.otype.v(n),
        E.oslots.s(n)[0],
        E.oslots.s(n)[-1],
    ))

 109641: book          first slot=1     , last slot=13979 
 109695: chapter       first slot=13714 , last slot=13979 
 110998: verse         first slot=13964 , last slot=13979 
  13979: word          first slot=13979 , last slot=13979 


## Going down

We go to the chapters of the second book, and just count them.

In [18]:
columns = L.d(secondBook, otype='chapter')
print(len(columns))

16


## The first line
We pick the first verse and the first word, and explore what is above and below them.

In [20]:
firstVerse = L.d(firstBook, otype='verse')[0]

for n in [1, firstVerse]:
    indent(level=0)
    info('Node {}'.format(n), tm=False)
    indent(level=1)
    info('UP', tm=False)
    indent(level=2)
    info('\n'.join(['{:<15} {}'.format(u, F.otype.v(u)) for u in L.u(n)]), tm=False)
    indent(level=1)
    info('DOWN', tm=False)
    indent(level=2)
    info('\n'.join(['{:<15} {}'.format(u, F.otype.v(u)) for u in L.d(n)]), tm=False)
indent(level=0)
info('Done', tm=False)

Node 1
   |   UP
   |      |   109928          verse
   |      |   109668          chapter
   |      |   109641          book
   |   DOWN
   |      |   
Node 109928
   |   UP
   |      |   109668          chapter
   |      |   109641          book
   |   DOWN
   |      |   1               word
   |      |   2               word
   |      |   3               word
   |      |   4               word
   |      |   5               word
   |      |   6               word
   |      |   7               word
   |      |   8               word
Done


# Text

We print the original transcriptions

In [21]:
T.languages

{'': {'language': 'default', 'languageEnglish': 'default'}}

# Edge features: left and right

We have not talked about edges much. If the nodes correspond to the rows in the big spreadsheet,
the edges point from one row to another.

One edge we have encountered: the special feature `oslots`.
Each non-slot node is linked by `oslots` to all of its slot nodes.

An edge is really a feature as well.
Whereas a node feature is a column of information,
one cell per node, 
an edge feature is also a column of information, one cell per pair of nodes.

In the tablets quads may be subdivided into subquads and signs, related by operators.
If there is an operator `op` between `qLeft` and `qRight`, there is an 
edge between `qLeft` and `qRight` with value `op`.

# Next steps

By now you have an impression how to compute around in the Hebrew Bible.
While this is still the beginning, I hope you already sense the power of unlimited programmatic access
to all the bits and bytes in the data set.

Here are a few directions for unleashing that power.

## Search
Text-Fabric contains a flexible search engine, that does not only work for the BHSA data,
but also for data that you add to it.
There is a tutorial dedicated to [search](search.ipynb).
And if you already know MQL queries, you can build from that in
[searchFromMQL](searchFromMQL.ipynb).

## Explore additional data
The ETCBC has a few other repositories with data that work in conjunction with the BHSA data.
One of them you have already seen: 
[phono](https://github.com/ETCBC/phono),
for phonetic transcriptions.

There is also
[parallels](https://github.com/ETCBC/parallels)
for detecting parallel passages,
and
[valence](https://github.com/ETCBC/valence)
for studying patterns around verbs that determine their meanings.

## Add your own data
If you study the additional data, you can observe how that data is created and also
how it is turned into a text-fabric data module.
The last step is incredibly easy. You can write out every Python dictionary where the keys are numbers
and the values string or numbers as a Text-Fabric feature.
When you are creating data, you have already constructed those dictionaries, so writing
them out is just one method call.
See for example how the
[flowchart](https://github.com/ETCBC/valence/blob/master/programs/flowchart.ipynb#Add-sense-feature-to-valence-module)
notebook in valence writes out verb sense data.
![flow](images/valence.png)

You can then easily share your new features on GitHub, so that your colleagues everywhere 
can try it out for themselves.

## Export to Emdros MQL

[EMDROS](http://emdros.org), written by Ulrik Petersen,
is a text database system with the powerful *topographic* query language MQL.
The ideas are based on a model devised by Christ-Jan Doedens in
[Text Databases: One Database Model and Several Retrieval Languages](https://books.google.nl/books?id=9ggOBRz1dO4C).

Text-Fabric's model of slots, nodes and edges is a fairly straightforward translation of the models of Christ-Jan Doedens and Ulrik Petersen.

[SHEBANQ](https://shebanq.ancient-data.org) uses EMDROS to offer users to execute and save MQL queries against the Hebrew Text Database of the ETCBC.

So it is kind of logical and convenient to be able to work with a Text-Fabric resource through MQL.

If you have obtained an MQL dataset somehow, you can turn it into a text-fabric data set by `importMQL()`,
which we will not show here.

And if you want to export a Text-Fabric data set to MQL, that is also possible.

After the `Fabric(modules=...)` call, you can call `exportMQL()` in order to save all features of the
indicated modules into a big MQL dump, which can be imported by an EMDROS database.

# Clean caches

Text-Fabric pre-computes data for you, so that it can be loaded faster.
If the original data is updated, Text-Fabric detects it, and will recompute that data.

But there are cases, when the algorithms of Text-Fabric have changed, without any changes in the data, that you might
want to clear the cache of precomputed results.

There are two ways to do that:

* Locate the `.tf` directory of your dataset, and remove all `.tfx` files in it.
  This might be a bit awkward to do, because the `.tf` directory is hidden on Unix-like systems.
* Call `TF.clearCache()`, which does exactly the same.

It is not handy to execute the following cell all the time, that's why I have commented it out.
So if you really want to clear the cache, remove the comment sign below.

In [39]:
# TF.clearCache()